# Collect data

In [2]:
# import Lib
import pandas as pd
import numpy as np
import urllib.robotparser
import time
from bs4 import BeautifulSoup
import requests
from fractions import Fraction

### Check dữ liệu trên trang này có hợp pháp khi thu thập

In [3]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://store.steampowered.com/robots.txt')
rp.read()

url = 'https://store.steampowered.com/search/?ndl=1'
rp.can_fetch('*', url)

True

Dữ liệu hợp pháp!

### Crawl data với các thuộc tính

'''  
title  
genre  
tags  
category  
is_dlc  
is_mature  
franchise  
release date  
developer  
publisher  
languages  
overwhelmingly positive count  
user reviews count  
negative count  
price  
'''

In [4]:
# Title = []  
# Genre = []  
# Tags = []   
# withDLC = []  
# isMature = [] 
# Franchise = []
# ReleaseDate = []  
# Developer = []  
# Publisher = []
# Languages = []
# PositiveReviews = []
# TotalReviews = []
# NegativeReviews = []  
# OriginalPrice = []
# DiscountPercent = []

In [5]:
def crawl_one_game(item_url):
    response = requests.get(item_url)
    content = response.text
    if response.status_code == 200:
        soup = BeautifulSoup(content, 'html.parser')
        
        # set all attribute is NaN
        title = np.nan 
        genre = np.nan
        tags = np.nan
        with_dlc = np.nan 
        is_mature = np.nan
        franchise = np.nan
        release_date = np.nan
        developer = np.nan
        publisher = np.nan
        languages = np.nan
        positive_reviews = np.nan
        total_reviews = np.nan
        negative_reviews = np.nan

        # Check if a dlc
        is_dlc = soup.find('div',attrs={'class':'game_area_bubble game_area_dlc_bubble'})
        # Check if a soundtrack
        is_soundtrack = soup.find('div',attrs={'class':'game_area_bubble game_area_soundtrack_bubble'})
        
        if is_dlc or is_soundtrack:
            return title, genre, tags, with_dlc, is_mature, franchise, release_date, developer, publisher, languages, positive_reviews, total_reviews, negative_reviews

        # Check game has DLC
        try:
            with_dlc_tmp = soup.find('div',attrs={'id':'gameAreaDLCSection'})
            with_dlc = (True if with_dlc_tmp else False) 
        except:
            pass

        # Title
        try:
            title = soup.find('div',attrs={'class':'apphub_AppName'}).string.strip() 
        except:
            pass

        # Genere
        try:
            genre = [] # A game can have more than one genre
            genre_tmp = soup.find('div',attrs={'id':'genresAndManufacturer'}).find('span')
            for i in genre_tmp.find_all('a'):
                genre.append(i.text.strip())
                
            genre = ", ".join(genre)
        except:
            genre = np.nan
            pass

        # Tag
        try:
            tags = [] # A game can have more than one tag
            for i in soup.find('div',attrs={'class':'glance_tags popular_tags'}).find_all('a'):         
                if i.text.strip() != '+':
                    tags.append(i.text.strip())
                    
            tags = ", ".join(tags)
        except:
            tags = np.nan
            pass

        # Check Mature
        try:
            is_mature_tmp = soup.find('div',attrs={'id':'game_area_content_descriptors'})
            is_mature = (True if is_mature_tmp else False)
        except:
            pass

        # Developer, Publisher, Franchise
        try:
            for i in soup.find('div',attrs={'id':'genresAndManufacturer'}).find_all('div',attrs={'class':'dev_row'}):
                if i.find('b').text == 'Developer:':
                    developer = i.find('a').text
                elif i.find('b').text == 'Publisher:':
                    publisher = i.find('a').text
                elif i.find('b').text == 'Franchise:':
                    franchise = i.find('a').text
        except:
            pass

        # Release Date
        try:
            release_date = soup.find('div',attrs={'class':'date'}).string.strip()
        except:
            pass 

        # Languages
        try:
            languages = [] # A game can have more than one language
            for i in soup.find('div',attrs={'id':'languageTable'}).find_all('td',attrs={'class':'ellipsis'}):   
                languages.append(i.text.strip())
            
            languages = ", ".join(languages)
        except:
            languages = np.nan
            pass

        # All review
        try:
            total_reviews = soup.find('label',attrs={'for':'review_type_all'}).find('span',attrs={'class':'user_reviews_count'}).string.strip()
            total_reviews = int(total_reviews.replace(')','').replace('(',''))
        except:
            pass

        # Negative review
        try:
            negative_reviews = soup.find('label',attrs={'for':'review_type_negative'}).find('span',attrs={'class':'user_reviews_count'}).string.strip()
            negative_reviews = int(negative_reviews.replace(')','').replace('(',''))
        except:
            pass

        # Positive review
        try:
            positive_reviews = soup.find('label',attrs={'for':'review_type_positive'}).find('span',attrs={'class':'user_reviews_count'}).string.strip()
            positive_reviews = int(positive_reviews.replace(')','').replace('(',''))
        except:
            pass

        return title, genre, tags, with_dlc, is_mature, franchise, release_date, developer, publisher, languages, positive_reviews, total_reviews, negative_reviews

In [26]:
data = pd.read_csv('steam_game.csv')
data.tail()

,Title,Genre,Tags,withDLC,isMature,Franchise,ReleaseDate,Developer,Publisher,Languages,PositiveReviews,TotalReviews,NegativeReviews,OriginalPrice,DiscountPercent
6740,Aground,"Adventure, Indie, RPG","Exploration, Open World, Base Building, Surviv...",True,False,Fancy Fish Games,"17 Apr, 2020",Fancy Fish Games,Fancy Fish Games,"English, Portuguese - Brazil, Russian, German","(1,484)","(1,575)",91,"205.000,00₫",-66%
6741,TOKYO GHOUL:re [CALL to EXIST],Action,"Action, Anime, Violent, Gore, Character Custom...",False,True,NaN,"14 Nov, 2019",Three Rings Inc.,BANDAI NAMCO Entertainment,"English, French, Italian, German, Spanish - Sp...","(1,280)","(1,824)",544,"680.000,00₫",-90%
6742,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"527.000,00₫",-30%
6743,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"3.430.000,00₫",-56%
6744,Hexcells Infinite,"Casual, Indie, Strategy","Puzzle, Casual, Indie, Logic, Minimalist, Rela...",False,False,NaN,"1 Sep, 2014",Matthew Brown,Matthew Brown,"English, French, German, Spanish - Spain, Japa...","(3,910)","(4,028)",118,"73.500,00₫",-70%


In [27]:
flag_page=True
index=287

# get the games from all the pages and not give it a range because its very dynamic
while(flag_page and index!=400):
    Title = []  
    Genre = []  
    Tags = []   
    withDLC = []  
    isMature = [] 
    Franchise = []
    ReleaseDate = []  
    Developer = []  
    Publisher = []
    Languages = []
    PositiveReviews = []
    TotalReviews = []
    NegativeReviews = []  
    OriginalPrice = []
    DiscountPercent = []
    print(index)
    
    # Add page parameter
    parameter = '&page='+str(index)
    
    # Send a request to the website
    web_response = requests.get(url+parameter)
        
    # Get the content of the request
    web_content = web_response.text
    
    if web_response.status_code == 200: # request successfully
        web_soup = BeautifulSoup(web_content, 'html.parser')
        
        item_link = []
        
        # Check if the last page
        if index != 1:
            # Find the number of buttons on the page and return False if it has 2
            button = web_soup.find_all('a',attrs={'class':'pagebtn'})
            if len(button)!= 2 :
                flag_page = False
    
        # Add each link to links
        links = web_soup.find('div', attrs={'id': 'search_resultsRows'})
        if links is None:
            index += 1
            continue
    
        for link in links.find_all('a'):
            # Price with discount
            with_discount = link.find('div',attrs={'class':'discount_block search_discount_block'})
            # Price without discount
            without_discount = link.find('div',attrs={'class':'discount_block search_discount_block no_discount'})
            # Free price
            free_price = link.find('div',attrs={'class':'discount_block no_discount search_discount_block'})
            
            if with_discount:
                DiscountPercent.append(with_discount.find('div',attrs={'class':'discount_pct'}).text)
                OriginalPrice.append(with_discount.find('div',attrs={'class':'discount_original_price'}).text)      
            elif without_discount:
                DiscountPercent.append(np.nan)
                OriginalPrice.append(without_discount.find('div',attrs={'class':'discount_final_price'}).text)
            elif free_price:
                DiscountPercent.append(np.nan)
                OriginalPrice.append(free_price.find('div',attrs={'class':'discount_final_price free'}).text)
            else:
                break
        
            item_link.append(link['href'])
            
        # Add attribute of each game
        for item in item_link:    
            title, genre, tags, with_dlc, is_mature, franchise, release_date, developer, publisher, languages, positive_reviews, total_reviews, negative_reviews = crawl_one_game(item)              
            
            # Add values into attributes
            Title.append(title)  
            Genre.append(genre)
            Tags.append(tags)  
            withDLC.append(with_dlc)
            isMature.append(is_mature)
            Franchise.append(franchise)
            ReleaseDate.append(release_date) 
            Developer.append(developer)
            Publisher.append(publisher)
            Languages.append(languages)
            PositiveReviews.append(positive_reviews)
            TotalReviews.append(total_reviews)
            NegativeReviews.append(negative_reviews)
    crawled_data = pd.DataFrame({"Title": Title,
                     "Genre": Genre,
                     "Tags": Tags,
                     "withDLC": withDLC,
                     "isMature": isMature,
                     "Franchise": Franchise,
                     "ReleaseDate": ReleaseDate,
                     "Developer": Developer,
                     "Publisher": Publisher,
                     "Languages": Languages,
                     "PositiveReviews": PositiveReviews,
                     "TotalReviews": TotalReviews,
                     "NegativeReviews": NegativeReviews,
                     "OriginalPrice": OriginalPrice,
                     "DiscountPercent": DiscountPercent})   
                     
    data = pd.concat([data, crawled_data], axis = 0) 
    # Increase page index
    index += 1

287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399


In [28]:
# data = pd.DataFrame({"Title": Title,
#                      "Genre": Genre,
#                      "Tags": Tags,
#                      "withDLC": withDLC,
#                      "isMature": isMature,
#                      "Franchise": Franchise,
#                      "ReleaseDate": ReleaseDate,
#                      "Developer": Developer,
#                      "Publisher": Publisher,
#                      "Languages": Languages,
#                      "PositiveReviews": PositiveReviews,
#                      "TotalReviews": TotalReviews,
#                      "NegativeReviews": NegativeReviews,
#                      "OriginalPrice": OriginalPrice,
#                      "DiscountPercent": DiscountPercent})

data

,Title,Genre,Tags,withDLC,isMature,Franchise,ReleaseDate,Developer,Publisher,Languages,PositiveReviews,TotalReviews,NegativeReviews,OriginalPrice,DiscountPercent
0,Baldur's Gate 3,"Adventure, RPG, Strategy","RPG, Choices Matter, Story Rich, Character Cus...",True,True,Baldur's Gate,"3 Aug, 2023",Larian Studios,Larian Studios,"English, French, German, Spanish - Spain, Poli...","(491,707)","(508,265)","(16,558)","990.000,00₫",-10%
1,Counter-Strike 2,"Action, Free to Play","FPS, Shooter, Multiplayer, Competitive, Action...",True,True,NaN,"21 Aug, 2012",Valve,Valve,"English, Czech, Danish, Dutch, Finnish, French...","(6,840,193)","(7,802,330)","(962,137)",Free,NaN
2,Lethal Company,"Action, Adventure, Indie, Early Access","Online Co-Op, Horror, First-Person, Psychologi...",False,True,NaN,"23 Oct, 2023",Zeekerss,Zeekerss,English,"(200,445)","(204,086)","(3,641)","142.000,00₫",NaN
3,Cyberpunk 2077,RPG,"Cyberpunk, Open World, Nudity, RPG, Singleplay...",True,True,NaN,"9 Dec, 2020",CD PROJEKT RED,CD PROJEKT RED,"English, French, Italian, German, Spanish - Sp...","(561,816)","(695,253)","(133,437)","990.000,00₫",-50%
4,ELDEN RING,"Action, RPG","Souls-like, Dark Fantasy, Open World, RPG, Dif...",False,True,Bandai Namco Entertainment,"24 Feb, 2022",FromSoftware Inc.,FromSoftware Inc.,"English, French, Italian, German, Spanish - Sp...","(684,531)","(741,142)","(56,611)","898.000,00₫",-40%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"589.000,00₫",-65%
21,AER Memories of Old,"Adventure, Indie","Adventure, Exploration, Indie, Relaxing, Open ...",True,False,NaN,"25 Oct, 2017",Forgotten Key,Daedalic Entertainment,"English, German, Spanish - Spain, French, Japa...","(5,784)","(6,717)",933,"165.000,00₫",-90%
22,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2.840.000,00₫",-54%
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"102.000,00₫",-80%


In [29]:
data.to_csv('steam_game.csv', index = False)